In [ ]:
from datasets import load_dataset

dataset = load_dataset("QuyenAnhDE/Diseases_Symptoms")

In [ ]:
dataset_org = load_dataset("gretelai/symptom_to_diagnosis")

In [ ]:
dataset_org

In [ ]:
#show the number of Name column which is not duplicated
#show that unique element of dataset["train"]['Name'] which is list type
labels= dataset_org["train"]['output_text']

unique_names = set(labels)


In [ ]:
len(unique_names)

In [ ]:
dataset

In [ ]:
#show the number of Name column which is not duplicated
#show that unique element of dataset["train"]['Name'] which is list type
labels= dataset["train"]['Name']

unique_names = set(labels)

len(unique_names)

In [ ]:
dataset = dataset["train"]

In [ ]:
dataset[2]

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Zabihin/Symptom_to_Diagnosis")

def preprocess_function(examples):
    # Tokenizing the text
    result = tokenizer(examples['Symptoms'], padding='max_length', truncation=True, max_length=512)
    # Assuming "Code" is what you want to predict and it's categorical data
    if 'Code' in examples:
        result["labels"] = [label_to_id[label] for label in examples['Name']]  # Convert labels to numerical IDs
    return result

# Assuming `dataset` is your loaded dataset
# Convert label strings to integers (if they aren't already)
label_list = dataset['Name']  # Adjust based on your actual dataset structure
label_to_id = {label: i for i, label in enumerate(set(label_list))}
id_to_label = {i: label for label, i in label_to_id.items()}

# Map the preprocessing function over the whole dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)



In [ ]:
from transformers import AutoModelForSequenceClassification, AutoConfig, AutoTokenizer
from transformers import AutoModel
import torch

# Load the base model
model_base = AutoModelForSequenceClassification.from_pretrained("Zabihin/Symptom_to_Diagnosis",from_tf=True)

# Replace the classifier for PyTorch (assuming the base model is similar to BERT)
model_base.classifier = torch.nn.Linear(model_base.config.hidden_size, 392)


config = AutoConfig.from_pretrained("Zabihin/Symptom_to_Diagnosis")
config.num_labels = 392  # Update the number of labels in the configuration


In [ ]:
# import torch.nn as nn

# # Assuming the model's classifier is a single linear layer
# # This replaces it with a new linear layer with 392 output features
# model.config.num_labels = len(set(label_list))
# model.classifier = nn.Linear(model.config.hidden_size, 392)


In [ ]:
#model.config.num_labels = 392
model = model_base

In [ ]:
print(model)

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}


In [ ]:
from datasets import DatasetDict

# Split the dataset into training and testing
train_test_split = tokenized_dataset.train_test_split(test_size=0.15)  # 80% training, 20% testing

# Split the test set into validation and test
validation_test_split = train_test_split['test'].train_test_split(test_size=0.5)  # Splits the 20% into two parts of 10% each

# Now, assemble the final tokenized_datasetDict including all splits
split_dataset = DatasetDict({
    'train': train_test_split['train'],
    'validation': validation_test_split['train'],  # Using 'train' here because it's the training part of the split
    'test': validation_test_split['test']
})


In [ ]:
split_dataset

In [ ]:
tokenized_dataset

In [ ]:

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./results',          # Directory for saving logs and model checkpoints
    evaluation_strategy='epoch',     # Evaluation is done at the end of each epoch
    learning_rate=2e-5,              # Fine-tuning learning rate
    per_device_train_batch_size=8,   # Batch size per device during training
    per_device_eval_batch_size=8,    # Batch size for evaluation
    num_train_epochs=3,              # Total number of training epochs
    weight_decay=0.01,               # Strength of weight decay regularization
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,  # Assuming your dataset has a train split
    eval_dataset=tokenized_dataset,  # Assuming your dataset has a validation split
    compute_metrics=compute_metrics,  # Define your own function to compute metrics
)


In [ ]:
# Assuming 'labels' is the field in your dataset containing the target labels
unique_labels = set(tokenized_dataset['Name'])
num_labels = len(unique_labels)
print("Number of unique labels:", num_labels)


In [ ]:
trainer.train()


In [ ]:
trainer.evaluate(tokenized_dataset['test'])  # Assuming your dataset has a test split
